<a href="https://colab.research.google.com/github/vishalgimhan/NLP-Projects/blob/main/Twitter_Sentiment_Analysis_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [247]:
#install kaggle library
#! pip install kaggle

In [248]:
#configure the pathe of Kaggle .json file
!mkdir -p ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [249]:
#Import Twitter dataset

In [250]:
#API to fetch dataset from Kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [251]:
#Extracting the compressed dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('Dataset is extracted')

Dataset is extracted


In [252]:
#import the dependencies
import numpy as np
import pandas as pd                         #to convert csv to dataframe
import re                                   #regular expressions
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer  #Stemming
#from nltk.stem import WordNetLemmaitizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle

In [253]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [254]:
#print the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [255]:
#loading dataset
#twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', random_state=42)

Data Preprocessing

In [256]:
#Number of rows and columns
twitter_data.shape

(20000, 7)

In [257]:
#Explore dataset
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,1,2186553499,Mon Jun 15 19:00:18 PDT 2009,NO_QUERY,jazzjeet,watching run's house!,watch run hous
1,0,1989072482,Mon Jun 01 00:15:19 PDT 2009,NO_QUERY,CeciliaYong,has to do her HB assignment and journals for D...,hb assign journal dcp
2,0,1993556896,Mon Jun 01 10:34:02 PDT 2009,NO_QUERY,mikejcarroll,finished setting up his 24-inch LED monitor an...,finish set inch led monitor macbook pro quot w...
3,0,2252261182,Sat Jun 20 05:51:45 PDT 2009,NO_QUERY,_Jamesage,so gutted. victor matfield was 1 day away from...,gut victor matfield day away move saint decid ...
4,1,2187575464,Mon Jun 15 20:29:09 PDT 2009,NO_QUERY,FaBuFun,@priscillacita oh yeah; I know what that's lik...,priscillacita oh yeah know like quot quot jean


In [258]:
#Specifying columns and rows
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')

In [259]:
#Filter out 20000 sample from the large dataset

# Filter data by target class
df_zeros = twitter_data[twitter_data['target'] == 0]
df_ones = twitter_data[twitter_data['target'] == 4]

# Resample dataframes to get 10000 samples for each class
df_zeros_resampled = df_zeros.sample(10000, replace=True, random_state=42)
df_ones_resampled = df_ones.sample(10000, replace=True, random_state=42)

# Concatenate the resampled dataframes
balanced_data = pd.concat([df_zeros_resampled, df_ones_resampled])

# Shuffle the data to ensure random distribution of classes
balanced_data = balanced_data.sample(frac=1, random_state=42)

# Reset the index after concatenating
balanced_data.reset_index(drop=True, inplace=True)

# Print the class distribution
print(f"Class distribution: {np.bincount(balanced_data['target'])}")

twitter_data = balanced_data

Class distribution: [10000     0     0     0 10000]


In [260]:
twitter_data.shape

(20000, 6)

In [261]:
twitter_data.head()

,target,id,date,flag,user,text
0,4,2186553499,Mon Jun 15 19:00:18 PDT 2009,NO_QUERY,jazzjeet,watching run's house!
1,0,1989072482,Mon Jun 01 00:15:19 PDT 2009,NO_QUERY,CeciliaYong,has to do her HB assignment and journals for D...
2,0,1993556896,Mon Jun 01 10:34:02 PDT 2009,NO_QUERY,mikejcarroll,finished setting up his 24-inch LED monitor an...
3,0,2252261182,Sat Jun 20 05:51:45 PDT 2009,NO_QUERY,_Jamesage,so gutted. victor matfield was 1 day away from...
4,4,2187575464,Mon Jun 15 20:29:09 PDT 2009,NO_QUERY,FaBuFun,@priscillacita oh yeah; I know what that's lik...


In [262]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  20000 non-null  int64 
 1   id      20000 non-null  int64 
 2   date    20000 non-null  object
 3   flag    20000 non-null  object
 4   user    20000 non-null  object
 5   text    20000 non-null  object
dtypes: int64(2), object(4)
memory usage: 937.6+ KB


In [263]:
twitter_data.describe()

,target,id
count,20000.00000,2.000000e+04
mean,2.00000,2.001893e+09
std,2.00005,1.914279e+08
min,0.00000,1.467841e+09
25%,0.00000,1.957310e+09
50%,2.00000,2.002398e+09
75%,4.00000,2.177701e+09
max,4.00000,2.329088e+09


In [264]:
#Check for missing values
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [265]:
#Check distribution of Target column
twitter_data['target'].value_counts()

#if uneven, [erform upsampling, downsampling techniques

target
4    10000
0    10000
Name: count, dtype: int64

In [266]:
#Making Target to 0 for Negative and 1 for Positive
twitter_data.replace({"target":{4:1}}, inplace=True)  #inplace True, to make changes to original dataset

#Text Processing

In [267]:
print(twitter_data['text'])

0                                   watching run's house! 
1        has to do her HB assignment and journals for D...
2        finished setting up his 24-inch LED monitor an...
3        so gutted. victor matfield was 1 day away from...
4        @priscillacita oh yeah; I know what that's lik...
                               ...                        
19995    listening to vegas skies. i love @symphnysldr ...
19996    @MarketingVeep Put that sledgehammer down, wom...
19997    @Funbrunette Awesome! I appreciate it! And by ...
19998                            Going going back to Cali 
19999    i have the will to stay up  and im most def pr...
Name: text, Length: 20000, dtype: object


In [268]:
#Stemming
port_stem = PorterStemmer()

In [269]:
def stemming(content):
  #sub module from regular expression library
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content) #remove(^) all words which are not with alphabets
  stemmed_content = stemmed_content.lower()           #lowercase all words
  stemmed_content = stemmed_content.split()           #split to list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #stem all words which are not stopwords
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [228]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [229]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,1,2186553499,Mon Jun 15 19:00:18 PDT 2009,NO_QUERY,jazzjeet,watching run's house!,watch run hous
1,0,1989072482,Mon Jun 01 00:15:19 PDT 2009,NO_QUERY,CeciliaYong,has to do her HB assignment and journals for D...,hb assign journal dcp
2,0,1993556896,Mon Jun 01 10:34:02 PDT 2009,NO_QUERY,mikejcarroll,finished setting up his 24-inch LED monitor an...,finish set inch led monitor macbook pro quot w...
3,0,2252261182,Sat Jun 20 05:51:45 PDT 2009,NO_QUERY,_Jamesage,so gutted. victor matfield was 1 day away from...,gut victor matfield day away move saint decid ...
4,1,2187575464,Mon Jun 15 20:29:09 PDT 2009,NO_QUERY,FaBuFun,@priscillacita oh yeah; I know what that's lik...,priscillacita oh yeah know like quot quot jean


In [230]:
print(twitter_data['stemmed_content'])

0                                           watch run hous
1                                    hb assign journal dcp
2        finish set inch led monitor macbook pro quot w...
3        gut victor matfield day away move saint decid ...
4           priscillacita oh yeah know like quot quot jean
                               ...                        
19995    listen vega sky love symphnysldr beauti voic l...
19996    marketingveep put sledgehamm woman u know much...
19997               funbrunett awesom appreci way still dm
19998                                      go go back cali
19999                               stay im def proud haha
Name: stemmed_content, Length: 20000, dtype: object


In [231]:
#Seperating the data and label
X = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [232]:
print(X)

['watch run hous' 'hb assign journal dcp'
 'finish set inch led monitor macbook pro quot work yay cours want inch led monitor home'
 ... 'funbrunett awesom appreci way still dm' 'go go back cali'
 'stay im def proud haha']


In [233]:
#Splitting data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y) #Stratify to give equal proportion of 0 and 1

In [234]:
print(X.shape, X_train.shape, X_test.shape)

(20000,) (16000,) (4000,)


In [235]:
# VEctorize - Convert textual data to numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train) #fit and transform
X_test = vectorizer.transform(X_test)       #only transform

In [236]:
print(X_train)

  (0, 12141)	0.6097468020427849
  (0, 19882)	0.5139921924392137
  (0, 12677)	0.6033414153778334
  (1, 4315)	0.1679143942945321
  (1, 20340)	0.1776771324390661
  (1, 10637)	0.2535743298411376
  (1, 6976)	0.15996406765202315
  (1, 10398)	0.17957737067437476
  (1, 10652)	0.21408161587457225
  (1, 18899)	0.2965327058987683
  (1, 5555)	0.3699249066176777
  (1, 19876)	0.36631197370261753
  (1, 3731)	0.39425389616837897
  (1, 13730)	0.45080056600500745
  (1, 7530)	0.23377414218203463
  (2, 13661)	0.2674058527389857
  (2, 12037)	0.25702280983029185
  (2, 19093)	0.1945681294273811
  (2, 2279)	0.26098061634899977
  (2, 17833)	0.2095368698777474
  (2, 2200)	0.2794934271811301
  (2, 8334)	0.18079678992293063
  (2, 16773)	0.3474261410626123
  (2, 10625)	0.3220342133128129
  (2, 18923)	0.35904672930796117
  :	:
  (15997, 17361)	0.20832911358757203
  (15997, 1331)	0.19315558148587147
  (15997, 16079)	0.19795489063511001
  (15997, 6818)	0.16513050728646772
  (15997, 18627)	0.21740397958379745
  (15998

Logistic Regression Model

In [237]:
#training the model
model = LogisticRegression(max_iter=1000)

In [238]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [239]:
#Accuracy on training data

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(y_train, X_train_prediction)

In [240]:
print('Accuracy Score on training data: ', training_data_accuracy)

Accuracy Score on training data:  0.8564375


In [241]:
#Accuracy on test data

X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(y_test, X_test_prediction)

In [242]:
print('Accuracy Score on test data: ', test_data_accuracy)

Accuracy Score on test data:  0.74475


Save the Model

In [243]:
import pickle

In [244]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb')) #wb to write file

In [245]:
#using the saved model

#loading the model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb')) #rb to read file

In [246]:
X_new = X_test[200]
print(y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("Negative Tweet")
else:
  print("Positive Tweet")

0
[1]
Positive Tweet
